In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=1000, edgeitems=30)
pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 2000)

In [ ]:
# Some StackOverflow magic to enable importing of local module
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from multiLevelCoSurrogates.local import base_dir

In [ ]:
df = pd.read_csv(base_dir+'records.csv', index_col='index')
df.head()

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(9,12))

grouped = df.groupby(by=['which_model', 'fidelity'])

for ax, group in zip(axes.flatten(), grouped):
    name, sub_df = group
    sub_df = sub_df.groupby(by='iteration').mean()
    sub_df.plot(y=['mse_high', 'mse_low', 'mse_diff'], ax=ax)
    ax.set_title('acq on {}, {} fidelity updated'.format(*name))

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(9,12))

grouped = df.groupby(by=['which_model', 'fidelity'])

mse_names = ['mse_high', 'mse_low', 'mse_diff']

for ax, group in zip(axes.flatten(), grouped):
    name, sub_df = group

    # Separate out the MSE values after initial training to use as reference
    num_iters = len(sub_df['iteration'].unique())
    first_rows = sub_df.loc[0::num_iters,mse_names].values
    
    # Get all values and reshape without copying s.t. the iterations are a separate dimension
    values = sub_df.loc[:,mse_names].values
    old_shape = values.shape
    new_shape = old_shape[0]//num_iters, num_iters, len(mse_names)
    values.shape = new_shape
    
    # Now we can use numpy's internal broadcasting for efficient division, and return the old shape afterwards
    new_values = values / first_rows[:,None,:]
    new_values.shape = old_shape

    # Finally the values are returned to the dataframe and plotted
    sub_df.loc[:,mse_names] = new_values
    sub_df = sub_df.groupby(by='iteration').mean()
    sub_df.plot(y=mse_names, ax=ax)
    ax.set_title('acq on {}, {} fidelity updated'.format(*name))

plt.tight_layout()
plt.show()